In [1]:
pip install pyserial


You should consider upgrading via the 'c:\Users\PC\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

for p in ports:
    print(p.device, "-", p.description)


COM4 - Lien série sur Bluetooth standard (COM4)
COM6 - Lien série sur Bluetooth standard (COM6)
COM8 - Lien série sur Bluetooth standard (COM8)
COM9 - Lien série sur Bluetooth standard (COM9)
COM3 - Lien série sur Bluetooth standard (COM3)
COM10 - Lien série sur Bluetooth standard (COM10)
COM5 - Lien série sur Bluetooth standard (COM5)
COM7 - Lien série sur Bluetooth standard (COM7)
COM13 - Périphérique série USB (COM13)


In [12]:
import serial
import numpy as np
import time

# ==========================
# CONFIGURATION PORT SÉRIE
# ==========================
PORT = "COM13"   # 🔥 Mets ton port ici (COM3 / COM6 etc.)
BAUD = 921600    # Même débit que ton STM32

ser = serial.Serial(PORT, BAUD, timeout=0.01)

print(f"Port ouvert : {PORT} à {BAUD} baud")

# ==========================
# TEST DE LECTURE SIMPLE
# ==========================
print("Lecture en cours... (CTRL + STOP LE KERNEL pour arrêter)")

while True:
    header = ser.read(1)

    # Si pas de header → on continue à écouter
    if header != b'\xAA':
        continue

    # Lecture des 12 bytes suivants (6 x int16)
    data = ser.read(12)

    # Si la trame est incomplète → on ignore
    if len(data) != 12:
        continue

    # Décodage binaire
    values = np.frombuffer(data, dtype=np.int16)

    print("Valeurs reçues :", values)


Port ouvert : COM13 à 921600 baud
Lecture en cours... (CTRL + STOP LE KERNEL pour arrêter)
Valeurs reçues : [-18178  -5633    255    256      0 -31744]
Valeurs reçues : [-19458  -7681  -3073    255   -512   3839]
Valeurs reçues : [-19970  -7681  -4097    511   -768   6143]
Valeurs reçues : [-18946  -9217  -3585     -1    255    512]
Valeurs reçues : [-8449 -8193 -6145  -769  -769  9727]
Valeurs reçues : [-18946  -7425   -513   -513   1023 -15872]
Valeurs reçues : [ -6913  -1537    255   -256 -19969 -20482]
Valeurs reçues : [-19714  -7937  -3841    255   -256 -19713]
Valeurs reçues : [-19714  -8193  -3585     -1    255  -9472]
Valeurs reçues : [ -7425  -2561     -1    767 -25344 -20226]
Valeurs reçues : [-19714  -6913  -1793    255      0 -18432]
Valeurs reçues : [12544  9216  4096  1024   512 24320]
Valeurs reçues : [-7169 -6657 -6657  -513  -257 17407]
Valeurs reçues : [-19970  -7937  -2049     -1   -257   4607]
Valeurs reçues : [ -6913  -1537    511    256 -19712 -18690]
Valeurs reçu

KeyboardInterrupt: 

In [17]:
import serial
import numpy as np
import joblib
import librosa
import time

# ============================
# CONFIG
# ============================
SR = 1024                  # fréquence d’échantillonnage
WINDOW_SIZE = 1024         # 1 seconde
HOP = 1024                 # pas = 1s (pas de recouvrement pour tester d’abord)
CHANNELS = 6

PORT = "COM13"              # adapte selon ton PC
BAUD = 921600

# === Paramètres MFCC identiques au preprocessing ===
N_MFCC = 20
N_MELS = 40
N_FFT = 256
HOP_LENGTH = 128
WIN_LENGTH = N_FFT
FMIN = 0.0
FMAX = 128.0

# ============================
# CHARGEMENT DU MODELE
# ============================
svm = joblib.load(r"C:\Users\PC\Documents\a_PROJET_5A\Projet_5A\training\models\svm_mfcc.joblib")
scaler = joblib.load(r"C:\Users\PC\Documents\a_PROJET_5A\Projet_5A\training\models\svm_scaler.joblib")

print("Modèle chargé.")


# ============================
# FONCTION EXTRACTION FEATURES
# ============================
def features_mfcc_window(window):
    """
    window : shape (1024, 6)
    Retourne un vecteur (240,) comme dans l’entraînement SVM
    """
    feats = []

    for ch in range(CHANNELS):
        y = window[:, ch].astype(float)

        mfcc = librosa.feature.mfcc(
            y=y,
            sr=SR,
            n_mfcc=N_MFCC,
            n_fft=N_FFT,
            hop_length=HOP_LENGTH,
            win_length=WIN_LENGTH,
            center=True,
            pad_mode="reflect",
            n_mels=N_MELS,
            fmin=FMIN,
            fmax=FMAX,
            dct_type=2,
            norm="ortho",
        )

        delta = librosa.feature.delta(mfcc, order=1)

        feats.extend(mfcc.mean(axis=1))
        feats.extend(delta.mean(axis=1))

    return np.array(feats)  # shape (240,)


# ============================
# OUVERTURE UART
# ============================
ser = serial.Serial(PORT, BAUD, timeout=0.01)
print(f"Port série {PORT} ouvert.")


# ============================
# BOUCLE TEMPS RÉEL
# ============================
buffer = np.zeros((WINDOW_SIZE, CHANNELS))
index = 0

print("=== Début acquisition temps réel ===")

while True:
    # Lecture du header
    head = ser.read(1)
    if head != b'\xAA':
        continue  # mauvais packet → on ignore

    # Lecture des 12 bytes suivants (6 x int16)
    data = ser.read(12)
    if len(data) != 12:
        continue

    # Conversion binaire → tableau Python
    values = np.frombuffer(data, dtype=np.int16)   # shape (6,)

    # Mise en buffer
    buffer[index] = values
    index += 1

    # Si une fenêtre complète est remplie
    if index >= WINDOW_SIZE:
        window = buffer.copy()

        # Extraction MFCC
        feat = features_mfcc_window(window)

        # Normalisation
        feat_scaled = scaler.transform([feat])

        # Prédiction
        pred = svm.predict(feat_scaled)[0]

        # Affichage
        print("→ Geste prédit :", pred)

        # Remet à zéro pour la prochaine fenêtre
        index = 0


Modèle chargé.
Port série COM13 ouvert.
=== Début acquisition temps réel ===
→ Geste prédit : 0
→ Geste prédit : 1


KeyboardInterrupt: 

In [15]:
import serial

ser = serial.Serial("COM13", 921600, timeout=0.5)

while True:
    data = ser.read(20)
    print(data)


b'\x00\x05\x00\x00\x00n\x01I\x00\x1e\x00\x0e\x00\x06\x00\x01\x00e\x01I'
b'\x00!\x00\x10\x00\x06\x00\x04\x00!\x01E\x00\x1f\x00\x12\x00\x06\x00\x05'
b'\x00\xd4\x009\x00\x1e\x00\x12\x00\x05\x00\x04\x00e\x00)\x00\x1d\x00\x10'
b'\x00\x05\x00\x04\x00\xe9\xff\x0b\x00\x1e\x00\x10\x00\x04\x00\x08\x00\x8a\xff\xef'
b'\xff\x07\x00\x0e\x00\x01\x00\x03\x00-\xff\xd9\xff\xfa\xff\r\x00\x04\x00\x06'
b'\x00\xd7\xfe\xc5\xff\xf3\xff\x0c\x00\x03\x00\x04\x00\x8f\xfe\xb4\xff\xed\xff\t'
b'\x00\x02\x00\x04\x00\x82\xfe\xaf\xff\xe8\xff\x01\x00\x03\x00\x03\x00\xa3\xfe\xb3'
b'\xff\xe4\xff\xfa\xff\x03\x00\x00\x00\xd8\xfe\xc0\xff\xe2\xff\xf5\xff\x04\x00\x02'
b'\x008\xff\xc6\xff\xe4\xff\xf2\xff\x04\x00\x03\x00\x98\xff\xe1\xff\xe4\xff\xf0'
b'\xff\x02\x00\x00\x00\x05\x00\xf7\xff\xf4\xff\xfa\xff\x04\x00\x02\x00\x84\x00\x19'
b'\x00\x05\x00\x00\x00\x04\x00\x03\x00\xc8\x003\x00\x12\x00\x05\x00\t\x00\x02'
b'\x00=\x01C\x00\x19\x00\x0b\x00\x07\x00\x02\x00\x8a\x01P\x00 \x00\r'
b'\x00\x08\x00\x05\x00\x81\x01Q\x00#\x00\x11\x00\t\

KeyboardInterrupt: 